##  [망고플레이트 크롤링](https://www.mangoplate.com/)    
### [참고 사이트](https://juwon2021.tistory.com/414)   
### 1 Page

### ** Window terminal에서
- conda activate   
- conda activate ml-env   
(ml-env) 환경 확인 후   
- jupyter notebook   
- conda install -c anaconda requests   
- conda install bs4   
- conda install lxml   
- conda install -c conda-forge selenium   
- conda install tqdm   
- connda install -c conda-forge folium   
- connda install -c conda-forge googlemaps   
- pip install chromedriver-autoinstaller   